<a href="https://colab.research.google.com/github/Kalyandask/FMML-Projects-and-labs/blob/main/AIML_project_Module2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
from Bio import SeqIO
from tqdm.autonotebook import tqdm
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
# --- Load Data ---
data = pd.read_csv("INDIA_685.csv")
print("Original Metadata Loaded:")
print(data.head())

# --- Step 1: TIME SERIES PREPARATION (Crucial Addition) ---
# NOTE: The original data may or may not have a 'Collection_Date'.
# We create a SYNTHETIC 'Date' column for demonstration, assuming the 685 samples
# were collected uniformly over 90 days (March to May 2020).
start_date = pd.to_datetime('2020-03-01')
end_date = pd.to_datetime('2020-05-30')
data['Date'] = pd.to_datetime(np.random.choice(pd.date_range(start_date, end_date), size=len(data)))
data = data.sort_values(by='Date').reset_index(drop=True)

print("\nMetadata with Synthetic Date:")
print(data[['StrainID', 'Date', 'State']].head())

In [ ]:
mutations = []
for i in range(len(data)):
    # Safely handle the string format '[mut1, mut2]' -> 'mut1, mut2'
    sample_muts_str = data["Nuc_muts"][i][1:][:-1]
    if sample_muts_str:
        sample_muts = sample_muts_str.split(",")
        for j in sample_muts:
            mut = j.strip()[1:][:-1]
            if(len(mut) >= 2 and mut not in mutations):
                mutations.append(mut)

print(f"\nTotal Unique Mutations: {len(mutations)}")

# Create the binary Mutation Profile DataFrame (rows=Strains, cols=Mutations)
samplelist = data["StrainID"].tolist()
mutation_df = pd.DataFrame(0, index=samplelist, columns=mutations)

for i in tqdm(range(len(data)), desc="Building Mutation Matrix"):
    strainid = data["StrainID"][i]
    sample_muts_str = data["Nuc_muts"][i][1:][:-1]
    if sample_muts_str:
        sample_muts = sample_muts_str.split(",")
        for j in sample_muts:
            mut = j.strip()[1:][:-1]
            if(len(mut) >= 2 and mut in mutation_df.columns):
                mutation_df.loc[strainid, mut] = 1

print("\nMutation Profile Matrix (Sample):")
print(mutation_df.head())

In [ ]:
# Select the key mutations to track their frequency
KEY_MUTATIONS = [
    'C13730T', # I/A3i subclade (Telangana/Delhi/TN)
    'C23929T', # I/A3i subclade (Telangana/Delhi/TN)
    'G25563T'  # I/GJ-20A subclade (Gujarat)
]

# Ensure the selected mutations are actually in the dataset
mutations_to_plot = [m for m in KEY_MUTATIONS if m in mutation_df.columns]
if not mutations_to_plot:
    print("\nError: Key mutations not found in the dataset for plotting.")
else:
    # Join the mutation profile with the date metadata
    df_plot = mutation_df.loc[data['StrainID']][mutations_to_plot]
    df_plot['Date'] = data['Date'].values

In [ ]:
df_plot['Total_Sequences'] = df_plot.index.to_series().rank(method='first').astype(int)

In [ ]:
weekly_data = df_plot.groupby(df_plot['Date'].dt.to_period('W')).agg(
        **{mut: ('Total_Sequences', 'count') for mut in mutations_to_plot},
        Total_Count=('Total_Sequences', 'count')
    ).fillna(0)

In [ ]:
for mut in mutations_to_plot:
        weekly_data[f'Frequency_{mut}'] = (weekly_data[mut] / weekly_data['Total_Count']) * 100

In [ ]:
weekly_data['Week'] = weekly_data.index.to_timestamp()
df_final_plot = weekly_data.filter(like='Frequency_')
df_final_plot['Week'] = weekly_data['Week']
df_final_plot = df_final_plot.reset_index(drop=True)
df_final_plot = df_final_plot.melt(id_vars=['Week'], var_name='Mutation', value_name='Frequency (%)')
df_final_plot['Mutation'] = df_final_plot['Mutation'].str.replace('Frequency_', '')

# --- Plotting ---
fig = px.line(
    df_final_plot,
    x='Week',
    y='Frequency (%)',
    color='Mutation',
    title='Temporal Prevalence of Key SARS-CoV-2 Mutations (Synthetic Data)',
    labels={'Week': 'Collection Week', 'Frequency (%)': 'Weekly Prevalence (%)'},
    line_shape='spline'
)

fig.update_layout(
    xaxis_title="Collection Week",
    yaxis_title="Weekly Prevalence (%)",
    legend_title="Mutation",
    hovermode="x unified"
)

fig.show(renderer="colab")